<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#数据准备" data-toc-modified-id="数据准备-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>数据准备</a></span></li><li><span><a href="#创建表项" data-toc-modified-id="创建表项-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>创建表项</a></span></li><li><span><a href="#查看配置" data-toc-modified-id="查看配置-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>查看配置</a></span></li><li><span><a href="#搜索" data-toc-modified-id="搜索-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>搜索</a></span></li><li><span><a href="#近义词模型" data-toc-modified-id="近义词模型-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>近义词模型</a></span></li><li><span><a href="#结合近义词搜索" data-toc-modified-id="结合近义词搜索-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>结合近义词搜索</a></span><ul class="toc-item"><li><span><a href="#测试" data-toc-modified-id="测试-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>测试</a></span></li><li><span><a href="#实现" data-toc-modified-id="实现-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>实现</a></span></li></ul></li></ul></div>

## Setup

In [79]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from datetime import datetime
import pprint
import gensim

In [3]:
es = Elasticsearch()

## 数据准备

In [74]:
!head /tmp/model_name.txt

CAT统计
获取上下行绑定的信道
RCC模板配置与配置检查
attenuation&equalizer
通话结束后准入带宽占用查询
*Node_cb068
配置BPI+认证白名单
防MAC欺骗验证:env_init(),execute(),check(),test_destory(),OWFunc
配置证书管理状态:OWFunc;env_init();execute();check();test_destory()
查询CM版本信息


## 创建表项

In [55]:
test_score_index = "modle_index"
type_name = "my_type"
field_name = "content"

In [127]:
my_data_settting = {
    "mappings": {
        type_name: {
            "properties": {
                field_name: {
                    "type": "text",
                    "analyzer": "ik_max_word",
                    "search_analyzer": "ik_smart"
                }
            }
        }
    }
}

In [128]:
es.indices.delete(index=test_score_index)
es.indices.create(index=test_score_index, body=my_data_settting)

{'acknowledged': True}

{'acknowledged': True, 'index': 'modle_index', 'shards_acknowledged': True}

In [129]:
def decode_nginx_log(data_iter):
    for idx, row in enumerate(data_iter):
        # Filter out the below from each log line
        # id_label = row
        # cmd_format_t = str(row['cmd_format_t'])

        # Index for elasticsearch. Typically timestamp.
        # idx = ...

        es_fields_keys = (field_name,)
        es_fields_vals = (row,)

        # We return a dict holding values from each line
        es_nginx_d = dict(zip(es_fields_keys, es_fields_vals))

        # Return the row on each iteration
        yield idx, es_nginx_d   # <- Note the usage of 'yield'

def es_add_bulk():
    with open("/tmp/model_name.txt", "r") as f:
        data_iter = f.readlines()
        data_iter = [x.strip() for x in data_iter]

    # NOTE the (...) round brackets. This is for a generator.
    k = ({
            "_index": test_score_index,
            "_type" : type_name,
            "_id"   : idx,
            "_source": es_nginx_d,
         } for idx, es_nginx_d in decode_nginx_log(data_iter))

    helpers.bulk(es, k)

# Now, just run it.
es_add_bulk()

## 查看配置

In [130]:
pprint.pprint(es.indices.get_mapping(index=test_score_index))
pprint.pprint(es.indices.get_settings(index=test_score_index))

{'modle_index': {'mappings': {'my_type': {'properties': {'content': {'analyzer': 'ik_max_word',
                                                                     'search_analyzer': 'ik_smart',
                                                                     'type': 'text'}}}}}}
{'modle_index': {'settings': {'index': {'creation_date': '1525338542356',
                                        'number_of_replicas': '1',
                                        'number_of_shards': '5',
                                        'provided_name': 'modle_index',
                                        'uuid': 'c2GQxIJnQue2Rowzos_kdA',
                                        'version': {'created': '6020399'}}}}}


In [131]:
es.search(index=test_score_index, body={"query": {"match_all": {}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '0',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': 'CAT统计'},
    '_type': 'my_type'},
   {'_id': '14',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置EQAM信道模板'},
    '_type': 'my_type'},
   {'_id': '19',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置上行信道绑定的频谱管理策略组'},
    '_type': 'my_type'},
   {'_id': '22',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置RX光机的光路AGC增益控制范围'},
    '_type': 'my_type'},
   {'_id': '24',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '清除准入控制统计'},
    '_type': 'my_type'},
   {'_id': '25',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': 'CM-REG-STATUS模板'},
    '_type': 'my_type'},
   {'_id': '26',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置负载均衡策略'},
    '_

## 搜索

In [132]:
es.search(index=test_score_index, explain = False,size=3, body={
    "query": {
        "match": {
            field_name: {
                "query": "配置 sfid"
            }
        }
    },
    "highlight": {
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '37',
    '_index': 'modle_index',
    '_score': 4.8841333,
    '_source': {'content': '获取动态流sfid信息'},
    '_type': 'my_type',
    'highlight': {'content': ['获取动态流<em>sfid</em>信息']}},
   {'_id': '620',
    '_index': 'modle_index',
    '_score': 2.126432,
    '_source': {'content': '配置vlan'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>配置</em>vlan']}},
   {'_id': '2',
    '_index': 'modle_index',
    '_score': 2.1227982,
    '_source': {'content': 'RCC模板配置与配置检查'},
    '_type': 'my_type',
    'highlight': {'content': ['RCC模板<em>配置</em>与<em>配置</em>检查']}}],
  'max_score': 4.8841333,
  'total': 192},
 'timed_out': False,
 'took': 6}

## 近义词模型

In [84]:
bin_file = '/root/workspace/mount/code/gensim/med256_5.model.bin'
word_vectors = gensim.models.Word2Vec.load(bin_file)

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [92]:
"设置" in word_vectors.wv.vocab

True

In [96]:
word_vectors.wv.most_similar("设置", topn=10)

[('修改', 0.523277759552002),
 ('设定', 0.39470696449279785),
 ('配置', 0.37758272886276245),
 ('更改', 0.35820212960243225),
 ('指定', 0.3255549371242523),
 ('小时数', 0.30822598934173584),
 ('为', 0.30258968472480774),
 ('调整', 0.3020206093788147),
 ('取值', 0.2891148030757904),
 ('无穷大', 0.28566598892211914)]

In [99]:
[x[0] for x in word_vectors.wv.most_similar("设置", topn=10) if len(x[0]) > 1]

['修改', '设定', '配置', '更改', '指定', '小时数', '调整', '取值', '无穷大']

## 结合近义词搜索

### 测试

In [136]:
es.search(index=test_score_index, explain = False,size=1, body={
    "query": {
        "match": {
            field_name: {
                "query": "设置  验证"
            }
        }
    },
    "highlight": {
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '255',
    '_index': 'modle_index',
    '_score': 8.095947,
    '_source': {'content': '严格组设置及验证'},
    '_type': 'my_type',
    'highlight': {'content': ['严格组<em>设置</em>及<em>验证</em>']}}],
  'max_score': 8.095947,
  'total': 17},
 'timed_out': False,
 'took': 2}

In [135]:
es.search(index=test_score_index, explain = False,size=1, body={
    "query": {
        "match": {
            field_name: {
                "query": "设定  验证"
            }
        }
    },
    "highlight": {
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '718',
    '_index': 'modle_index',
    '_score': 5.3301387,
    '_source': {'content': 'NIT验证参数'},
    '_type': 'my_type',
    'highlight': {'content': ['NIT<em>验证</em>参数']}}],
  'max_score': 5.3301387,
  'total': 14},
 'timed_out': False,
 'took': 3}

In [137]:
' '.join([x[0] for x in word_vectors.wv.most_similar("验证", topn=10) if len(x[0]) > 1])
' '.join([x[0] for x in word_vectors.wv.most_similar("设定", topn=10) if len(x[0]) > 1])

'测试 上网 正常 定量 状态刷新 ipsubnet-vlan 检查 svrp 可靠性测试 正确处理'

'设置 aucmodifyfldlist 报文大小 接近 -tc 值未 _NUM_pps 文件大小 绝对 温度'

In [134]:
es.search(index=test_score_index, explain = False,size=1, body={
    "query": {
        "bool": {
            "should": [
                {"match": { field_name: {"query": "设定  验证", "boost": 2}}},
                {"match": { field_name: "设置 aucmodifyfldlist 报文大小 接近 -tc 值未 _NUM_pps 文件大小 绝对 温度 测试 上网 正常 定量 状态刷新 ipsubnet-vlan 检查 svrp 可靠性测试 正确处理"}},
            ]
        }
    },
    "highlight": {
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '255',
    '_index': 'modle_index',
    '_score': 11.838544,
    '_source': {'content': '严格组设置及验证'},
    '_type': 'my_type',
    'highlight': {'content': ['严格组<em>设置</em>及<em>验证</em>']}}],
  'max_score': 11.838544,
  'total': 98},
 'timed_out': False,
 'took': 9}

### 实现

In [139]:
input_str_a = "设定验证"
input_str_b = "设置验证"

In [166]:
def analyze_with_ik(i_str):
    ik_analyze = es.indices.analyze(body={"text": i_str, "analyzer": "ik_smart"})
    return [x['token'] for x in ik_analyze['tokens']]

In [168]:
analyze_with_ik(input_str_a)
analyze_with_ik(input_str_b)

['设定', '验证']

['设置', '验证']

In [169]:
def get_similar_word(i_list):
    ret_list = []
    for v in i_list:
        ret_list.extend([x[0] for x in word_vectors.wv.most_similar(v, topn=10) if len(x[0]) > 1])
    return ret_list

In [179]:
def get_similar_search_wrod(i_str):
    analyze_list = analyze_with_ik(i_str)
    similar_list = get_similar_word(analyze_list)
    analyze_similar_list = analyze_with_ik(' '.join(similar_list))
    return set(similar_list) & set(analyze_similar_list)

In [182]:
get_similar_search_wrod(input_str_a)

{'aucmodifyfldlist',
 'ipsubnet-vlan',
 'svrp',
 '上网',
 '定量',
 '接近',
 '文件大小',
 '检查',
 '正常',
 '正确处理',
 '测试',
 '温度',
 '绝对',
 '设置'}

In [188]:
def search_api(i_str):   
    ret = es.search(index=test_score_index, explain = False,size=1, body={
        "query": {
            "bool": {
                "should": [
                    {"match": { field_name: {"query": i_str, "boost": 2}}},
                    {"match": { field_name: ' '.join(get_similar_search_wrod(i_str))}},
                ]
            }
        },
        "highlight": {
            "fields" : {
                field_name : {}
            }
        }
    })
    return ret

In [189]:
search_api(input_str_a)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '255',
    '_index': 'modle_index',
    '_score': 11.838544,
    '_source': {'content': '严格组设置及验证'},
    '_type': 'my_type',
    'highlight': {'content': ['严格组<em>设置</em>及<em>验证</em>']}}],
  'max_score': 11.838544,
  'total': 68},
 'timed_out': False,
 'took': 13}

In [190]:
search_api(input_str_b)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '255',
    '_index': 'modle_index',
    '_score': 16.191895,
    '_source': {'content': '严格组设置及验证'},
    '_type': 'my_type',
    'highlight': {'content': ['严格组<em>设置</em>及<em>验证</em>']}}],
  'max_score': 16.191895,
  'total': 264},
 'timed_out': False,
 'took': 5}